In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib
import tensorflow as tf
import tensorflow.keras as keras
import pickle
import os
import time
import sys
import warnings

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, optimizers, layers, metrics, models, regularizers

warnings.filterwarnings("ignore")

In [2]:
# Read the dataset and store it in a dataframe
df1 = pd.read_csv("Data/silo_current.csv")
df2 = pd.read_csv("Data/silo_history.csv")

df = pd.concat([df1,df2])

In [3]:
# Drop unnecessary columns
to_drop = ["...2", "Postcode", "Locatie", "Telefoon 1", "2e telefoonnummer", "Land", "...8", 
           "Verkooporganisatie", "...10", "Aanmaakdatum order", "Aanmaaktijd order", "Artikel", "...16", 
           "Planartikel", "...18", "Prod.hiër. niv.3", "Prod.hiër. niv.4", "Prod.hiër. niv.5", "ABC-teken", 
           "Opmerking 1 code", "Opmerking 2 code", "Opmerking 3 code", "Opmerking 4 code", "Opmerking 5 code", 
           "Srt verk.doc.", "...33", "Verkoopleider", "Bedrijfsadviseur", "Vestiging", "Mogelijke leverdata", 
           "Gecreëerd door"]

df = df.drop(to_drop, axis=1)

In [4]:
# Change the column names to English
col_names = ["recipient", "silo_nr", "delivery_date", "prod_hier_1", "prod_hier_2",
            "prod_group", "prod_group_name", "delivery_nr", "kg_delivered", "kg_ordered"]

for idx, val in enumerate(df.columns):
    df.rename(columns={val:col_names[idx]}, inplace=True)

In [5]:
# Remove periods from delivery_date column and turn it into a datetime datatype
df["delivery_date"] = df["delivery_date"].replace({'\.':''}, regex=True)
df["delivery_date"] = pd.to_datetime(df["delivery_date"], format="%d%m%Y")

In [6]:
recipients = []

for i in df["recipient"]:
    if i not in recipients:
        recipients.append(i)
        
print(len(recipients))

6327


In [7]:
# Get the previous date for each order and add it to the dataframe
starting_df = pd.DataFrame(columns = col_names)

for i in recipients:
    test = df.loc[df["recipient"] == i].sort_values(by=["delivery_date"])
    test.reset_index(drop=True, inplace=True)
    
    test_unique = test.drop_duplicates(subset = ["delivery_date"])
    test_unique.reset_index(drop=True, inplace=True)
    
    for i in range(len(test_unique)):
        if i != 0:
            test_unique.loc[i,("prev_date")] = test_unique.loc[i-1,("delivery_date")]
        else:
            test_unique.loc[i,("prev_date")] = None

    test_unique.loc[:, ("prev_date")] = pd.to_datetime(test_unique.loc[:,("prev_date")], format="%d%m%Y")
    
    for i in range(len(test)):
        for j in range(len(test_unique)):
            if test.loc[i,("delivery_date")] == test_unique.loc[j,("delivery_date")]:
                test.loc[i,("prev_date")] = test_unique.loc[j,("prev_date")]
                
    starting_df = pd.concat([starting_df, test])
    
df = starting_df

In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
df.to_pickle("df.pkl")

In [11]:
df = pd.read_pickle("Data/df.pkl")

In [12]:
# Create the difference feature
df["difference"] = df.loc[:,("delivery_date")]-df.loc[:,("prev_date")]

In [13]:
# Turn any non-numeric silo number into a nan
df["silo_nr"] = [np.nan if silo.isnumeric() == False else silo for silo in df["silo_nr"]]

In [14]:
# Turn difference into an integer instead of a TimeDelta 
df["difference"] = [diff.days for diff in df["difference"]]

In [15]:
df.to_pickle("df_new.pkl")

In [16]:
df = pd.read_pickle("Data/df_new.pkl")

In [17]:
# Drop any row without a previous date
df.drop(df[pd.isna(df.prev_date) == True].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [18]:
# Get the first date in the dataset to be used as the base date
min_date = np.min(df["delivery_date"])

In [19]:
# Turn delivery date into an integer based on the number of days it is from the base date
df["delivery_date"] = [(date-min_date).days for date in df["delivery_date"]]

In [20]:
# Turn previous date into an integer based on the number of days it is from the base date
df["prev_date"] = [(date-min_date).days for date in df["prev_date"]]

In [21]:
print(df.dtypes)

recipient           object
silo_nr             object
delivery_date        int64
prod_hier_1         object
prod_hier_2         object
prod_group          object
prod_group_name     object
delivery_nr        float64
kg_delivered        object
kg_ordered          object
prev_date            int64
difference         float64
dtype: object


In [22]:
# Turn all columns into the right type
df["recipient"] = df["recipient"].astype("float32")
df["silo_nr"] = df["silo_nr"].astype("float32")
df["delivery_date"] = df["delivery_date"].astype("float32")
df["prod_hier_1"] = df["prod_hier_1"].astype("category")
df["prod_hier_2"] = df["prod_hier_2"].astype("category")
df["prod_group"] = df["prod_group"].astype("float32")
df["prod_group_name"] = df["prod_group_name"].astype("category")
df["delivery_nr"] = df["delivery_nr"].astype("float32")
df["kg_delivered"] = df["kg_delivered"].astype("float32")
df["kg_ordered"] = df["kg_ordered"].astype("float32")
df["prev_date"] = df["prev_date"].astype("float32")
df["difference"] = df["difference"].astype("float32")

In [23]:
print(df.dtypes)

recipient           float32
silo_nr             float32
delivery_date       float32
prod_hier_1        category
prod_hier_2        category
prod_group          float32
prod_group_name    category
delivery_nr         float32
kg_delivered        float32
kg_ordered          float32
prev_date           float32
difference          float32
dtype: object


In [24]:
# Drop any row without a silo number
df.drop(df[pd.isna(df.silo_nr) == True].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [25]:
# Create an array of all product hierarchy 1's
prod_hier_1 = df["prod_hier_1"]
prod_hier_1 = np.asarray(prod_hier_1)

In [26]:
# One-hot encode the product category level 1 feature
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(prod_hier_1)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

In [27]:
encoded = pd.DataFrame(onehot_encoded, columns=["RU Geit", "RU Grond", "RU Kalv", 
                                                "RU Melkv", "RU Vlvee", "Rundvee"])

In [28]:
# Add the one-hot encoded feature to the dataframe
result = pd.concat([df, encoded], axis=1)
df = result

In [29]:
df.to_pickle("df_final1.pkl")